## Direct messaging
In the previous notebook we saw how to setup an agent and send messages to it.  

Let's now create two agents and let's have one sending a message to the other and get a response back.

In [12]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from rich import print

Well' use a set of messages for our workflow communication

In [2]:
@dataclass
class RequestMessage:
    content: str

@dataclass
class ResponseMessage:
    content: str

@dataclass
class PoemMessage:
    content: str

@dataclass
class ReviewMessage:
    content: str

Time to create our two simple agents: PoemAgen will create a Poem while ReviewAgent will approve it.

In [ ]:
class ReviewAgent(RoutedAgent):
    @message_handler
    async def on_my_message(self, message: PoemMessage, ctx: MessageContext) -> ReviewMessage:
        print(f"[cyan]ReviewAgent received message: '{message.content}', reviewing...[/cyan]")
        return ReviewMessage(content="I like it! approved")


class PoemAgent(RoutedAgent):
    def __init__(self, description: str, reviewer_agent_type: str):
        super().__init__(description)
        self.reviewer_id = AgentId(reviewer_agent_type, self.id.key)

    @message_handler
    async def on_my_message(self, message: RequestMessage, ctx: MessageContext) -> ResponseMessage:
        print(f"[yellow]CoderAgent received message: '{message.content}'[/yellow]")

        # Send a direct message to the review agent and receives a response from it
        response = await self.send_message(PoemMessage(content="This is my wonderful poem"), recipient=self.reviewer_id)
        if isinstance(response, ReviewMessage):
            print(f"[yellow]CoderAgent received response from ReviewAgent: {response.content}[/yellow]")
            return ResponseMessage(content="'This is my wonderful poem' was reviewed and approved.")
        else:
            return ResponseMessage(content="Ops! Something went wrong")

As before, it's time to create the runtime, set all the necessary registration, run it and send a message to the PoemAgent.

In [11]:
# Create the runtime
runtime = SingleThreadedAgentRuntime()

# Register the agents with the runtime, note how we pass the type of review agent to the PoemAgent so it can send messages to it
await PoemAgent.register(runtime, type="poem_agent", factory=lambda: PoemAgent("I am the PoemAgent, I write poems for free", "review_agent"))
await ReviewAgent.register(runtime, type="review_agent", factory=lambda: ReviewAgent("I am the ReviewAgent, I review poems for (almost) free"))

runtime.start()
print("[magenta]Sending a message to the PoemAgent[/magenta]")
response=await runtime.send_message(RequestMessage(content="Please me a poem"), recipient=AgentId("poem_agent","default"))
print(response.content)
await runtime.stop_when_idle()
await runtime.close()

Sending a message to the PoemAgent

CoderAgent received message: 'Please me a poem'

ReviewAgent received message: 'This is my wonderful poem', reviewing...

CoderAgent received response from ReviewAgent: I like it! approved

'This is my wonderful poem' was reviewed and approved

As you might notice, direct messaging support request/response making syncronous communication between runtime and agents effortless.